# Сравнение GCNConv сеть с случайным подбрасыванием монетки на данных VK Cup

In [1]:
import torch

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html

!pip install torch-geometric


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 7.9 MB 32.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 27.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 2.4 MB 31.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 17.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689857 sha256=fe81a8f8ec9cf3f59c98d771c749f195b270f9d62072e2b9a8cdd57c0eb98

In [2]:
import numpy as np 
import pandas as pd 

df = pd.read_csv('/content/drive/MyDrive/train_mod.csv')
df

,Unnamed: 0,u,v,t,h
0,0,0,1,27,6
1,1,2,3,13,8
2,2,4,5,74,2
3,3,6,7,36,9
4,4,8,9,52,2
...,...,...,...,...,...
17414505,17414505,67072,94867,60,0
17414506,17414506,991066,1279987,71,0
17414507,17414507,125201,68549,67,0
17414508,17414508,27914,3234444,49,0


In [3]:
N_TOTAL = max(df['u'].max(), df['v'].max()) + 1
N_TOTAL

3234445

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_test.to_csv('test.csv', index=False)
del df_test
del df

In [5]:
def apk(pred, target, k):
    if len(pred) >= k:
        pred = pred[:k]

    ans, cnt = 0, 0
    tot = min(len(target), k) 
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            cnt += 1
            ans += cnt / (i + 1)
            s.add(pred[i])
    return ans / tot


def mapk(pred, target, k):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += apk(cur_pred, cur_target, k)
    return sum_metric / len(pred) 


In [6]:
df_test = pd.read_csv('/content/test.csv')

target_arr = [[] for _ in range(N_TOTAL)]

for i in range(df_test.shape[0]):
    u, v = df_test['u'].iloc[i], df_test['v'].iloc[i]
    target_arr[u].append(v)
    target_arr[v].append(u)

del df_test

## 1. Подбрасывание монетки

In [7]:
from tqdm.auto import trange

edges_set = set()
for i in trange(df_train.shape[0]):
    edges_set.add((df_train['u'].iloc[i], df_train['v'].iloc[i]))

  0%|          | 0/13931608 [00:00<?, ?it/s]

In [9]:
import random

K = 100
sum_m, cnt = 0, 0

for i in trange(N_TOTAL):
    if len(target_arr[i]) == 0:
        continue
    cur = []
    while len(cur) != K:
        pred = random.randint(0, N_TOTAL - 1)
        if (pred, i) not in edges_set and (i, pred) not in edges_set:
            cur.append(pred)
    sum_m += apk(cur, target_arr[i], K)
    cnt += 1

sum_m / cnt

  0%|          | 0/3234445 [00:00<?, ?it/s]

2.1530003953586344e-06

## Сетка из GCNConv:

In [7]:
import networkx as nx
G_train = nx.Graph()

G_train.add_nodes_from(list(range(N_TOTAL)))

G_train.add_edges_from(df_train[['u', 'v']].values)

In [8]:
import random
from networkx import ego_graph

class GraphGenerator:
    def __init__(self, G):
        self.G = G
        self.num_nodes = len(G.nodes)
        self.indices = np.arange(0, self.num_nodes)

    def get_random_subgraph(self, size_graph):
        mask = np.random.choice(self.indices, size=size_graph)
        return self.G.subgraph(self.indices[mask])

    def get_random_connected(self, thr=60):
        v0 = random.randint(0, N_TOTAL - 1)
        g0 = ego_graph(self.G, n=v0, radius=1)
        if len(g0.nodes) < thr:
            g0 = ego_graph(self.G, n=v0, radius=2)
        return g0

    def generate_random_graph(self, p_connected, size_graph, thr=60):
        if random.random() < p_connected:
            return self.get_random_connected(thr)
        else:
            return self.get_random_subgraph(size_graph)

In [9]:
from torch_geometric.utils.convert import to_networkx, from_networkx

gen = GraphGenerator(G_train)
g_sample = gen.generate_random_graph(0.5, 10)

In [10]:
g_sample.edges

EdgeView([(665985, 322096), (678786, 322096), (287239, 41435), (287239, 195168), (287239, 245189), (96525, 2356564), (163726, 245189), (517648, 624454), (517648, 245189), (1803794, 245189), (390034, 322096), (390034, 617471), (812179, 322096), (77204, 418329), (77204, 187314), (77204, 506187), (77204, 1337245), (77204, 2356564), (1344278, 322096), (672280, 1943354), (672280, 1577786), (1008665, 1903857), (1008665, 2923939), (1008665, 322096), (1008665, 1943354), (1008665, 2356564), (1008665, 245189), (1008665, 1831549), (418329, 322096), (1236121, 322096), (1337245, 245189), (1337245, 173613), (1086495, 322096), (1086495, 195168), (3000225, 1903857), (111269, 322096), (335270, 322096), (387114, 322096), (173613, 245189), (322095, 606532), (322095, 322096), (322096, 2974393), (322096, 617471), (322096, 1577786), (322096, 2550997), (322096, 209227), (322096, 210046), (322096, 187314), (322096, 1570161), (322096, 48760), (322096, 195168), (322096, 1389244), (322096, 606532), (322096, 1929

In [11]:
from_networkx(g_sample).edge_index

tensor([[ 0,  1,  2,  2,  2,  3,  4,  5,  5,  6,  7,  7,  8,  9,  9,  9,  9,  9,
         10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 14, 15, 15, 15, 16, 16,
         17, 18, 19, 20, 21, 22, 22, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24,
         24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25,
         25, 26, 27, 28, 28, 28, 29, 29, 30, 31, 32, 32, 33, 33, 34, 35, 35, 35,
         35, 35, 35, 35, 35, 35, 35, 35, 35, 36, 36, 37, 37, 38, 39, 39, 40, 41,
         41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42, 43, 44, 45, 45, 45, 46,
         47, 47, 48, 48, 48, 49, 49, 49, 49, 50, 50, 51, 52, 52, 52, 52, 53, 54,
         55, 56, 57, 57, 58, 59],
        [24, 24, 45, 49, 35, 41, 35, 37, 35, 35, 24, 47, 24, 13, 25, 39, 15, 41,
         24, 28, 29, 52, 18, 24, 28, 41, 35, 57,  9, 24, 24,  9, 35, 22, 24, 49,
         52, 12, 24, 24, 24, 15, 35, 36, 24,  0,  1,  7,  8, 10, 12, 13, 14, 16,
         19, 20, 21, 23, 27, 47, 29, 44, 38, 58, 25, 53, 56, 49, 31, 36, 51

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
device

device(type='cpu')

In [13]:
import torch 
import torch.nn as nn
import torch_geometric as tg
from torch_geometric.utils.convert import to_networkx, from_networkx

class GCNModel(nn.Module):
    def __init__(self, num_vertexes, embedding_dim, hidden_dim, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(num_vertexes, embedding_dim)
        self.layers = nn.ModuleList([])
        self.layers.append(tg.nn.GCNConv(embedding_dim, hidden_dim))
        self.relu = nn.ReLU()
        for i in range(num_layers):
            self.layers.append(tg.nn.GCNConv(hidden_dim, hidden_dim))
        self.last = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, graph):
        input_data = from_networkx(graph)
        input_vertexes = torch.tensor(list(graph.nodes)).to(device)
        hidden = self.embedding(input_vertexes)
        for layer in self.layers:
            hidden = layer(hidden, input_data.edge_index.to(device))
            hidden = self.relu(hidden)
        return self.last(hidden)

In [14]:
class BPRLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, logits, mask_positive):
        positive_scores = logits[mask_positive].reshape(-1, 1)
        negative_scores = logits[~mask_positive].reshape(1, -1)
        sum_loss = 0
        m1 = torch.ones(1, negative_scores.shape[1]).to(device)
        m2 = torch.ones(positive_scores.shape[0], 1).to(device)

        diff = -torch.log(self.sigmoid((positive_scores @ m1) - (m2 @ negative_scores)))
        sum_loss = torch.sum(diff)

        sum_loss /= len(positive_scores)
        sum_loss /= len(negative_scores)
        return sum_loss


In [15]:
model = GCNModel(N_TOTAL, 16, 64, 4).to(device)
criterion = BPRLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

In [ ]:
if True:
    model = torch.load("/content/drive/MyDrive/start_model.pt")

In [ ]:
from tqdm.auto import trange

iterations = 1000
batch_size = 16
save_every = 100

pbar = trange(iterations)  
for iteration in pbar:
    optimizer.zero_grad()
    input_graph = nx.Graph(gen.generate_random_graph(1, 60))
    vertexes = list(input_graph.nodes)
    id2pos = {val: pos for pos, val in enumerate(vertexes)}
    test_edges = []

    for i in range(batch_size):
        v0 = random.choice(vertexes)
        v1 = random.choice(vertexes)
        ITERATIONS = 100
        while v1 == v0 or (v0, v1) in test_edges or (v1, v0) in test_edges:
            v0 = random.choice(vertexes)
            v1 = random.choice(vertexes)
            ITERATIONS -= 1
            if ITERATIONS == 0:
                break
        if ITERATIONS == 0:
            break
        test_edges.append((v0, v1))
  
    if len(test_edges) == 0:
        continue
    labels = torch.tensor([edge in input_graph.edges for edge in test_edges]).to(device)
    if sum(labels) == 0 or sum(labels) == len(labels):
        continue
    input_graph.remove_edges_from(test_edges)
    output = model(input_graph)
    pred_logits = torch.cat([torch.sum(output[id2pos[v0]] * output[id2pos[v1]]).reshape(-1) for v0, v1 in test_edges], 0)
    loss = criterion(pred_logits, labels) 
    pbar.set_description(f"train loss: {loss.item()}")
    loss.backward()
    optimizer.step()

    if iteration % save_every == save_every - 1:
        torch.save(model, "/content/drive/MyDrive/start_model.pt")
    

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
embeddings = model(G_train).cpu().detach().numpy()

In [ ]:
!pip install nearpy

In [ ]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
from nearpy.distances import CosineDistance
from nearpy.filters import NearestFilter
import numpy as np

rbp = RandomBinaryProjections('default', 10)
engine = Engine(embeddings.shape[1], lshashes=[rbp], distance=CosineDistance(), vector_filters=[NearestFilter(300)])
for i in range(embeddings.shape[0]):
    engine.store_vector(embeddings[i], i)

In [ ]:
K = 100
sum_m, cnt = 0, 0

for i in trange(N_TOTAL):
    if len(target_arr[i]) == 0:
        continue
    cur = []
    while len(cur) != K:
        k_find = 2 * K
        pred = engine.neighbours(embeddings[i])[:k_find]
        if (pred, i) not in edges_set and (i, pred) not in edges_set:
            cur.append(pred)
    sum_m += apk(cur, target_arr[i], K)
    cnt += 1

sum_m / cnt